In [1]:
import pandas as pd
import numpy as np
import timm
import torchvision.transforms as T

from wildlife_tools.features import SuperPointExtractor, SiftExtractor, AlikedExtractor, DiskExtractor, DeepFeatures
from wildlife_tools.similarity import CosineSimilarity, MatchLOFTR, MatchLightGlue
from wildlife_tools.data.dataset import ImageDataset
from wildlife_tools.similarity.wildfusion import SimilarityPipeline, WildFusion
from wildlife_tools.similarity.calibration import IsotonicCalibration

metadata = {'metadata':  pd.read_csv('../tests/TestDataset/metadata.csv'), 'root': '../tests/TestDataset'}
transform = T.Compose([T.Resize([224, 224]), T.ToTensor()])
dataset = ImageDataset(**metadata, transform=transform)


# WildFusion
- Run wildfusion as mean of
    - LightGlue + SuperPoint
    - LightGlue + ALIKED
    - LightGlue + DISK
    - LightGlue + SIFT
    - LOFTR
    - Deep features of MegaDescriptor-L

Additionally, shortlist is created using MegaDescriptor-L scores. It can be used to significantly speed up the matching pipelines by using only `B` samples from database per query, which are selected based on score in the shortlist.

In [7]:
pipelines = [

    SimilarityPipeline(
        matcher = MatchLightGlue(features='superpoint'),
        extractor = SuperPointExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLightGlue(features='aliked'),
        extractor = AlikedExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLightGlue(features='disk'),
        extractor = DiskExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLightGlue(features='sift'),
        extractor = SiftExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLOFTR(pretrained='outdoor'),
        extractor = None,
        transform = T.Compose([
            T.Resize([512, 512]),
            T.Grayscale(),
            T.ToTensor(),
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = CosineSimilarity(),
        extractor = DeepFeatures(
            model = timm.create_model('hf-hub:BVRA/wildlife-mega-L-384', num_classes=0, pretrained=True)
        ),
        transform = T.Compose([
            T.Resize(size=(384, 384)),
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]),
        calibration = IsotonicCalibration()
    ),
]


priority_pipeline =  SimilarityPipeline(
    matcher = CosineSimilarity(),
    extractor = DeepFeatures(
        model = timm.create_model('hf-hub:BVRA/wildlife-mega-L-384', num_classes=0, pretrained=True)
    ),
    transform = T.Compose([
        T.Resize(size=(384, 384)),
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]),
)

wildfusion = WildFusion(calibrated_pipelines = pipelines, priority_pipeline = priority_pipeline)

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (BVRA/wildlife-mega-L-384)
/mnt/appl/software/timm/1.0.8-foss-2023b-CUDA-12.4.0/lib/python3.11/site-packages/timm/models/_hub.py:190: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Plea

In [3]:
wildfusion.fit_calibration(dataset, dataset)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.02s/it]


### Basic WildFusion
- Run for all pairs
- Note that there are ones at a diagonal at the query and database that are the same datasets.

In [4]:
wildfusion(dataset, dataset)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.92s/it]


array([[1.00000000e+00, 8.41624476e-01, 8.53761989e-02, 2.58286972e-03],
       [8.45939177e-01, 1.00000000e+00, 2.93104603e-03, 1.28648551e-02],
       [5.72058793e-02, 2.93104603e-03, 1.00000000e+00, 6.89334981e-01],
       [3.01374998e-04, 1.28648551e-02, 6.97637682e-01, 1.00000000e+00]])

### Accelerated WildFusion

In [5]:
wildfusion(dataset, dataset, B=1)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.92s/it]


array([[  1., -inf, -inf, -inf],
       [-inf,   1., -inf, -inf],
       [-inf, -inf,   1., -inf],
       [-inf, -inf, -inf,   1.]], dtype=float32)

In [6]:
wildfusion(dataset, dataset, B=2)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.89s/it]


array([[1.        , 0.84016925,       -inf,       -inf],
       [0.85628253, 1.        ,       -inf,       -inf],
       [      -inf,       -inf, 1.        , 0.68457514],
       [      -inf,       -inf, 0.69002765, 1.        ]], dtype=float32)